In [1]:
import pandas as pd
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
captureSize = 35.8 #for tmb claculation
import seaborn as sns
sns.set(style = 'white', font_scale=1.5)

In [2]:
itrp_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/SKCM_v2/'
tcga_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/'
immu_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/InnateDB/'
nano_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/NanoString/'
ppi_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/PPI/'

merge_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/'

In [3]:
! ls -lh /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/*.TABLE

-rw-rw-r-- 1 was966 zitnik 3.4M Oct 12 15:03 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/GENE.TABLE
-rw-rw-r-- 1 was966 zitnik  75K Oct 12 15:25 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/NORMAL.PATIENTS.TABLE
-rw-rw-r-- 1 was966 zitnik 314M Oct 12 15:25 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/NORMAL.TPM.TABLE
-rw-rw-r-- 1 was966 zitnik 1.1M Oct 12 15:16 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/PATIENTS.TABLE
-rw-rw-r-- 1 was966 zitnik 4.4G Oct 12 15:04 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/TPM.TABLE


In [4]:
! ls -lh /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/SKCM_v2/*.TABLE

-rw-rw-r-- 1 was966 zitnik 55K Sep  1 14:56 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/SKCM_v2/PATIENTS.TABLE
-rw-rw-r-- 1 was966 zitnik 51M Sep  1 14:56 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/SKCM_v2/TPM_CORRECTION.TABLE
-rw-rw-r-- 1 was966 zitnik 51M Sep  1 14:56 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/SKCM_v2/TPM.TABLE


In [5]:
id2name = pd.read_pickle('/n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/GENE.TABLE').set_index('ensid_v36').gene_name
tcga_normal_x = pd.read_pickle('/n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/NORMAL.TPM.TABLE')
tcga_normal_x.columns = tcga_normal_x.columns.map(id2name)

In [6]:

## ITRP dataset
itrp_patient = pd.read_pickle(os.path.join(itrp_data_path, 'PATIENTS.TABLE'))
itrp_tpm = pd.read_pickle(os.path.join(itrp_data_path, 'TPM.TABLE'))
itrp_tpm_correct = pd.read_pickle(os.path.join(itrp_data_path, 'TPM_CORRECTION.TABLE'))

## TCGA dataset
tcga_gene = pd.read_pickle(os.path.join(tcga_data_path, 'GENE.TABLE'))
tcga_patient = pd.read_pickle(os.path.join(tcga_data_path, 'PATIENTS.TABLE'))
tcga_tpm = pd.read_pickle(os.path.join(tcga_data_path, 'TPM.TABLE'))

## Merge
ensid_freeman = pd.Series(dict(zip(itrp_tpm.index.map(lambda x:x.split('.')[0]), itrp_tpm.index)))
ensid_freeman.name = 'ensid_freeman'
common_genes = set(tcga_tpm.columns.map(lambda x:x.split('.')[0])) & set(itrp_tpm.index.map(lambda x:x.split('.')[0]))
common_genes = list(common_genes)
coomon_genes = pd.Series(common_genes).sort_values().tolist()
print(len(common_genes))

tcga_gene = tcga_gene.loc[common_genes]
tcga_gene = tcga_gene[tcga_gene.gene_type == 'protein_coding']
tcga_gene = tcga_gene.join(ensid_freeman)

tcga_tpm = tcga_tpm[tcga_gene.ensid_v36]
tcga_tpm.columns = tcga_gene.gene_name.tolist()

itrp_tpm = itrp_tpm.T[tcga_gene.ensid_freeman]
itrp_tpm.columns = tcga_gene.gene_name.tolist()

itrp_tpm_correct = itrp_tpm_correct.T[tcga_gene.ensid_freeman]
itrp_tpm_correct.columns = tcga_gene.gene_name.tolist()

itrp_patient = itrp_patient.loc[itrp_tpm.index]

18980


In [7]:
tcga_tpm.shape, itrp_tpm.shape, itrp_tpm_correct.shape, tcga_gene.shape

((10184, 18474), (334, 18474), (334, 18474), (18474, 5))

In [8]:
itrp_patient.shape, tcga_patient.shape

((334, 20), (10184, 18))

In [9]:
### remove genes with very low variance
def _sel_genes(x):
    var = x.var(axis=0) 
    sel_genes = var[var > 1e-8].index
    return sel_genes
sgs1 = itrp_tpm.join(itrp_patient.cohort).groupby('cohort').apply(_sel_genes)
var_gene_list = list(set.intersection(*map(set, sgs1)))
len(var_gene_list)

var = tcga_tpm.var() 
var_gene_list = var[var > 1e-4].index
len(var_gene_list)

18425

In [10]:
var_gene_list

Index(['C9orf153', 'ZNF610', 'PARD6B', 'ALPL', 'TRAM2', 'GPAT2', 'GALNT4',
       'NOL8', 'OR10J5', 'BCL7B',
       ...
       'CENPL', 'UBQLNL', 'RILPL1', 'KRTAP10-9', 'GCNT1', 'DOK3', 'CBS',
       'LRRC6', 'EIF4G2', 'LRRC10B'],
      dtype='object', length=18425)

## 01. Save 18425

In [11]:
tcga_tpm_1 = tcga_tpm[var_gene_list]
itrp_tpm_1 = itrp_tpm[var_gene_list]
itrp_tpm_1_correct =itrp_tpm_correct[var_gene_list]
tcga_tpm_normal = tcga_normal_x[var_gene_list]

gene_1 = tcga_gene[tcga_gene.gene_name.isin(var_gene_list)].reset_index().set_index('gene_name')
tcga_tpm_1.shape, tcga_tpm_normal.shape, itrp_tpm_1.shape, tcga_patient.shape, itrp_patient.shape, gene_1.shape

((10184, 18425),
 (714, 18425),
 (334, 18425),
 (10184, 18),
 (334, 20),
 (18425, 5))

In [12]:
! mkdir -p /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/18425
tcga_tpm_1.to_pickle(os.path.join(merge_data_path, '18425', 'TPM.TCGA.TABLE'))
tcga_tpm_normal.to_pickle(os.path.join(merge_data_path, '18425', 'TPM.TCGA.NORMAL.TABLE'))
itrp_tpm_1.to_pickle(os.path.join(merge_data_path, '18425', 'TPM.ITRP.TABLE'))
itrp_tpm_1_correct.to_pickle(os.path.join(merge_data_path, '18425', 'TPM.ITRP.TABLE.CORRECT'))

gene_1.to_pickle(os.path.join(merge_data_path, '18425', 'GENE.TABLE'))
tcga_patient.to_pickle(os.path.join(merge_data_path, '18425', 'PATIENT.TCGA.TABLE'))
itrp_patient.to_pickle(os.path.join(merge_data_path, '18425', 'PATIENT.ITRP.TABLE'))
! ls -lh /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/18425/*.TABLE

-rw-rw-r-- 1 was966 zitnik 1.4M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/18425/GENE.TABLE
-rw-rw-r-- 1 was966 zitnik  55K Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/18425/PATIENT.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik 1.1M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/18425/PATIENT.TCGA.TABLE
-rw-rw-r-- 1 was966 zitnik  48M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/18425/TPM.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik 101M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/18425/TPM.TCGA.NORMAL.TABLE
-rw-rw-r-- 1 was966 zitnik 1.4G Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/18425/TPM.TCGA.TABLE


## 02. Save 6575

In [13]:
# Immunogenes
df_immu_gene = pd.read_pickle(os.path.join(immu_data_path, 'InnateDB.TABLE'))
immu_gene_list = list(set(df_immu_gene.gene_name) & set(var_gene_list))
immu_gene_list = pd.Series(immu_gene_list).sort_values().tolist()

tcga_tpm_2 = tcga_tpm[immu_gene_list]
tcga_tpm_normal_2 = tcga_normal_x[immu_gene_list]

itrp_tpm_2 = itrp_tpm[immu_gene_list]
itrp_tpm_2_correct = itrp_tpm_correct[immu_gene_list]

gene_2 = tcga_gene[tcga_gene.gene_name.isin(immu_gene_list)].reset_index().set_index('gene_name')
mut_gene = df_immu_gene.set_index('gene_name')
mut_gene = mut_gene[mut_gene.columns[2:]]
gene_2 = gene_2.join(mut_gene)
tcga_tpm_2.shape, tcga_tpm_normal_2.shape, itrp_tpm_2.shape, tcga_patient.shape, itrp_patient.shape, gene_2.shape

((10184, 6575), (714, 6575), (334, 6575), (10184, 18), (334, 20), (6575, 11))

In [14]:
! mkdir -p /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/6575
tcga_tpm_2.to_pickle(os.path.join(merge_data_path, '6575', 'TPM.TCGA.TABLE'))
tcga_tpm_normal_2.to_pickle(os.path.join(merge_data_path, '6575', 'TPM.TCGA.NORMAL.TABLE'))

itrp_tpm_2.to_pickle(os.path.join(merge_data_path, '6575', 'TPM.ITRP.TABLE'))
itrp_tpm_2_correct.to_pickle(os.path.join(merge_data_path, '6575', 'TPM.ITRP.TABLE.CORRECT'))
gene_2.to_pickle(os.path.join(merge_data_path, '6575', 'GENE.TABLE'))
tcga_patient.to_pickle(os.path.join(merge_data_path, '6575', 'PATIENT.TCGA.TABLE'))
itrp_patient.to_pickle(os.path.join(merge_data_path, '6575', 'PATIENT.ITRP.TABLE'))
! ls -lh /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/6575/*.TABLE

-rw-rw-r-- 1 was966 zitnik 537K Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/6575/GENE.TABLE
-rw-rw-r-- 1 was966 zitnik  55K Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/6575/PATIENT.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik 1.1M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/6575/PATIENT.TCGA.TABLE
-rw-rw-r-- 1 was966 zitnik  17M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/6575/TPM.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik  36M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/6575/TPM.TCGA.NORMAL.TABLE
-rw-rw-r-- 1 was966 zitnik 512M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/6575/TPM.TCGA.TABLE


### 03. Save 735

In [15]:
# NanoString genes
df_nano_gene = pd.read_pickle(os.path.join(nano_data_path, 'NanoString.TABLE'))
nano_gene_list = list(set(df_nano_gene.index) & set(var_gene_list))
nano_gene_list = pd.Series(nano_gene_list).sort_values().tolist()
len(df_immu_gene), len(immu_gene_list), len(df_nano_gene), len(nano_gene_list)

(7394, 6575, 756, 735)

In [16]:
tcga_tpm_3 = tcga_tpm[nano_gene_list]
tcga_tpm_normal_3 = tcga_normal_x[nano_gene_list]

itrp_tpm_3 = itrp_tpm[nano_gene_list]
itrp_tpm_3_correct = itrp_tpm_correct[nano_gene_list]

gene_3 = tcga_gene[tcga_gene.gene_name.isin(nano_gene_list)].reset_index().set_index('gene_name')
gene_3 = gene_3.join(df_nano_gene)
tcga_tpm_3.shape, itrp_tpm_3.shape, tcga_patient.shape, itrp_patient.shape, gene_3.shape

((10184, 735), (334, 735), (10184, 18), (334, 20), (735, 10))

In [17]:
! mkdir -p /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/735
tcga_tpm_3.to_pickle(os.path.join(merge_data_path, '735', 'TPM.TCGA.TABLE'))
tcga_tpm_normal_3.to_pickle(os.path.join(merge_data_path, '735', 'TPM.TCGA.NORMAL.TABLE'))

itrp_tpm_3.to_pickle(os.path.join(merge_data_path, '735', 'TPM.ITRP.TABLE'))
itrp_tpm_3_correct.to_pickle(os.path.join(merge_data_path, '735', 'TPM.ITRP.TABLE.CORRECT'))

gene_3.to_pickle(os.path.join(merge_data_path, '735', 'GENE.TABLE'))
tcga_patient.to_pickle(os.path.join(merge_data_path, '735', 'PATIENT.TCGA.TABLE'))
itrp_patient.to_pickle(os.path.join(merge_data_path, '735', 'PATIENT.ITRP.TABLE'))
! ls -lh /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/735/*.TABLE

-rw-rw-r-- 1 was966 zitnik 101K Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/735/GENE.TABLE
-rw-rw-r-- 1 was966 zitnik  55K Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/735/PATIENT.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik 1.1M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/735/PATIENT.TCGA.TABLE
-rw-rw-r-- 1 was966 zitnik 4.2M Sep 13 22:50 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/735/TCGA.COR.TABLE
-rw-rw-r-- 1 was966 zitnik 4.2M Sep 13 22:50 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/735/TCGA.MIR.TABLE
-rw-rw-r-- 1 was966 zitnik 1.9M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/735/TPM.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik 4.1M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/735/TPM.TCGA.NORMAL.TABLE
-rw-rw-r-- 1 was966 zitnik  58M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITR

### 04. Save pRED Signatures (876)

In [18]:
df = pd.read_excel('./pRED CIT Signatures.xlsx')
df = df.iloc[1:]
df = df[df.Description != 'Internal']
df = df.set_index('Name')

def convert2dict(s):
    marker = s.iloc[4:].dropna().to_list()
    name = s.name
    type = s.Type
    return {'name':name, 'type':type, 'marker':marker}

df = df.apply(convert2dict, axis=1).apply(pd.Series).reset_index(drop=True).set_index('name')
df

,type,marker
name,,
CytotoxTcell_sc,Tcell,"[TRGC2, GZMH, KLRD1, FGFBP2, GZMB, PRF1, SAMD3..."
ExhCD8Tcell_sc,Tcell,"[PDCD1, HAVCR2, LAG3, ENTPD1, CD38, TOX]"
RegTcell_sc,Tcell,"[FOXP3, CCR8, PMCH, CCR4, RTKN2, CTLA4]"
Bcell_sc,Bcell,"[CD19, MS4A1, TNFRSF13C, VPREB3, PAX5, CR2]"
Myeloid_sc,Myeloid,"[CSF3R, MS4A6A, MS4A7, MNDA, C5AR1, FCGR2A, C3..."
...,...,...
Nivo_up_ROnly_Riaz17,PD_effect,"[PDCD1, SLC2A6, CXCR6, GZMH, MPO, MT1E, PRF1, ..."
NivoResponse_Riaz17,CPI_Response,"[MLC1, UBAP1L, PCDH11X, CLEC17A, P2RX5, IRAG2,..."
Endothelial_Kahn20,Endothelial / Vascular,"[ANGPT2, CDH5, ESAM, ESM1, ERG, ICAM2, TIE1]"


In [19]:
from itertools import chain
pRED_signs_list = list(set(chain(*df.marker.to_list())))
pRED_list = list(set(pRED_signs_list) & set(var_gene_list))
pRED_list = pd.Series(pRED_list).sort_values().tolist()
len(pRED_list)

876

In [20]:
tcga_tpm_4 = tcga_tpm[pRED_list]
tcga_tpm_normal_4 = tcga_normal_x[pRED_list]

itrp_tpm_4 = itrp_tpm[pRED_list]
itrp_tpm_4_correct = itrp_tpm_correct[pRED_list]

gene_4 = tcga_gene[tcga_gene.gene_name.isin(pRED_list)].reset_index().set_index('gene_name')
gene_4 = gene_4.join(df_nano_gene)
tcga_tpm_4.shape, itrp_tpm_4.shape, tcga_patient.shape, itrp_patient.shape, gene_4.shape

((10184, 876), (334, 876), (10184, 18), (334, 20), (876, 10))

In [21]:
! mkdir -p /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/876
tcga_tpm_4.to_pickle(os.path.join(merge_data_path, '876', 'TPM.TCGA.TABLE'))
tcga_tpm_normal_4.to_pickle(os.path.join(merge_data_path, '876', 'TPM.TCGA.NORMAL.TABLE'))

itrp_tpm_4.to_pickle(os.path.join(merge_data_path, '876', 'TPM.ITRP.TABLE'))
itrp_tpm_4_correct.to_pickle(os.path.join(merge_data_path, '876', 'TPM.ITRP.TABLE.CORRECT'))

gene_4.to_pickle(os.path.join(merge_data_path, '876', 'GENE.TABLE'))
tcga_patient.to_pickle(os.path.join(merge_data_path, '876', 'PATIENT.TCGA.TABLE'))
itrp_patient.to_pickle(os.path.join(merge_data_path, '876', 'PATIENT.ITRP.TABLE'))

df.to_pickle(os.path.join(merge_data_path, '876', 'pRED.TABLE'))

! ls -lh /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/876/*.TABLE

-rw-rw-r-- 1 was966 zitnik 116K Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/876/GENE.TABLE
-rw-rw-r-- 1 was966 zitnik  55K Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/876/PATIENT.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik 1.1M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/876/PATIENT.TCGA.TABLE
-rw-rw-r-- 1 was966 zitnik  13K Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/876/pRED.TABLE
-rw-rw-r-- 1 was966 zitnik 2.3M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/876/TPM.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik 4.8M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/876/TPM.TCGA.NORMAL.TABLE
-rw-rw-r-- 1 was966 zitnik  69M Oct 12 15:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/876/TPM.TCGA.TABLE
